# Подготовка файлов для кросскорреляции

Код, представленный ниже, используется для подсчета аффинностей гексамеров и их покрытия внутри каждого гена, отдельно ведутся подсчеты для прямой и обратной цепей.

Входные файлы:

511145_2.fasta - геном;

    aff_table.txt - таблица аффинностей гексамеров;

    plus.wig - покрытие прямой цепи;

    minus.wig - покрытие обратной цепи;

    coordinate.txt - координаты генов;

Получаемые файлы:

    affinity_all_FP1_plus_final.txt - аффинность по прямой цепи;

    affinity_all_FP1_minus_final.txt - аффинность по обратной цепи;

    coverage_all_FP1_plus_final.txt - покрытие по прямой цепи;

    coverage_all_FP1_minus_final.txt - покрытие по обратной цепи;

Основаная функция программы  - обработка гексамера. 
Функция принимает на вход сам гексамер, его начало и конец и переменную файла с покрытием.
На выходе - покрытие гексамера (сумма покрытия всех нуклеотидов гексамера из исходного файла с покрытием), аффинносить гексамера (берется из таблицы).

In [ ]:
def hexamer_processing(hex_seq, start, end, cover_file):#cover_file - список с покрытием каждой позиции.
    if hex_seq in aff_dict:
        affin = aff_dict[hex_seq]
    else:
        affin = 0
    cover = sum(list(map(float, cover_file[int(start) - 1 : int(end)])))
    return(affin, cover)

Подготовка файлов для работы:

In [ ]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC


with open('511145_2.fasta') as genome: #помним, что координаты будут нужны со старта до конца вклчительно
    line = genome.readlines()
    line1 = line[1:]
    line2 = []
    for i in line1:
        i = i.strip()
        line2.append(i)
    genome_seq = "".join(line2) #геном для работы с '+' генами
    a = Seq(genome_seq, IUPAC.unambiguous_dna)
    anti_genome_seq = str(a.reverse_complement()) #геном для работы с '-' генами
    affinity_for = [0] * len(genome_seq)#список с аффинностями гексамеров (для каждого записывается на третьей позиции)
    coverage_for = [0] * len(genome_seq)#список с покрытиями гексомеров (тоже на третьей позиции) - заполнится дальше
    affinity_rev = [0] * len(genome_seq)#список с аффинностями гексамеров (для каждого записывается на третьей позиции)
    coverage_rev = [0] * len(genome_seq)#список с покрытиями гексомеров (тоже на третьей позиции) - заполнится дальше

with open("aff_table.txt") as af_tab:
    aff_dict = {}
    lines = af_tab.readlines()
    for item in lines:
        aff_dict[item.split()[0]] = item.split()[1]

with open('plus.wig') as cover: #читаем файл с покрытием прямой цепи
    cover_f = cover.readlines()
    cover_f = cover_f[2:]
    cover_plus = []
    for i in cover_f:
        i = i.strip()
        cover_plus.append(i)
        
with open('minus.wig') as cover: #читаем файл с покрытием обратной цепи
    cover_f = cover.readlines()
    cover_f = cover_f[2:]
    cover_minus = []
    for i in cover_f:
        i = i.strip()
        cover_minus.append(i)

print("reading done")

Основная часть программы:

In [ ]:
with open('coordinate.txt', 'r') as coord:
    genes = coord.readlines()
    for k,item in enumerate(genes):            
        start, end = int(item.split()[0]), int(item.split()[1])
        if k%100 == 0:
            print(k)
        if "+" in item:
            current_gene = genome_seq[start: end] #теперь работаем с этим участком последовательности,
            #обрабатываем гексамеры в нем     
            c = 0 
            while c < len(current_gene) - 5:
                hexamer = current_gene[c: c + 6]
                hex_affin, hex_cover = hexamer_processing(hexamer, start + c, start + c + 6, cover_plus)
                    #а сейчас подсчет координаты третьего нуклеотида в гексомере
                affinity_for[start + c + 3] = hex_affin
                coverage_for[start + c + 3] = hex_cover
                c += 1
                    
        if "-" in item:
            start, end = len(anti_genome_seq) - end, len(anti_genome_seq) - start
            current_gene = anti_genome_seq[start : end] #теперь работаем с этим участком последовательности,
            #обрабатываем гексамеры в нем      
            c = 0 
            while c < len(current_gene) - 5:
                hexamer = current_gene[c: c + 6]
                hex_affin, hex_cover = hexamer_processing(hexamer, start + c, start + c + 6, cover_minus)
                affinity_rev[start + c + 3] = hex_affin
                coverage_rev[start + c + 3] = hex_cover
                c += 1

Запись данных в файлы:

In [ ]:
with open("affinity_all_FP1_plus_final.txt", "a") as af:
    for value in affinity_for:
        af.write(str(value) + "\n")
with open("coverage_all_FP1_plus_final.txt", "a") as co:
    for value in coverage_for:
        co.write(str(value) + "\n")
with open("affinity_all_FP1_minus_final.txt", "a") as af:
    for value in affinity_rev:
        af.write(str(value) + "\n")
with open("coverage_all_FP1_minus_final.txt", "a") as co:
    for value in coverage_rev:
        co.write(str(value) + "\n")